# Feature Engineering for Kalman Filter

Prepare data for hurricane track prediction using Kalman filter state-space model.


In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None


In [2]:
def load_ibtracs(path="ibtracs.ALL.list.v04r01.csv"):
    df = pd.read_csv(path, skiprows=[1], low_memory=False)
    df.columns = [c.lower() for c in df.columns]
    df = df.replace(' ', np.nan)
    
    df['storm_dir'] = pd.to_numeric(df['storm_dir'], errors='coerce').astype('Int64')
    df['storm_speed'] = pd.to_numeric(df['storm_speed'], errors='coerce').astype('Int64')
    df['usa_wind'] = pd.to_numeric(df['usa_wind'], errors='coerce').astype('Int64')
    df['usa_lat'] = pd.to_numeric(df['usa_lat'], errors='coerce').astype('Float64')
    df['usa_lon'] = pd.to_numeric(df['usa_lon'], errors='coerce').astype('Float64')
    df['iso_time'] = pd.to_datetime(df['iso_time'])
    
    return df

df = load_ibtracs("ibtracs.ALL.list.v04r01.csv")


In [3]:
# Filter to storms with sufficient observations for Kalman filter
obs_per_storm = df.groupby('sid').size()
valid_storms = obs_per_storm[obs_per_storm >= 2].index
df = df[df['sid'].isin(valid_storms)].copy()
df = df.sort_values(['sid', 'iso_time']).reset_index(drop=True)


In [4]:
def compute_velocity_from_positions(storm_data):
    """Compute velocity from position differences using haversine distance"""
    storm_data = storm_data.sort_values('iso_time').copy()
    
    dt = storm_data['iso_time'].diff().dt.total_seconds() / 3600  # hours
    dlat = storm_data['lat'].diff()
    dlon = storm_data['lon'].diff()
    
    # Handle longitude wrapping at ±180
    dlon = dlon.copy()
    dlon[dlon > 180] -= 360
    dlon[dlon < -180] += 360
    
    # Convert degrees to km (approximate)
    lat_km = dlat * 111.0
    lon_km = dlon * 111.0 * np.cos(np.radians(storm_data['lat']))
    
    # Compute speed (km/h) and direction (degrees)
    speed_kmh = np.sqrt(lat_km**2 + lon_km**2) / dt.replace(0, np.nan)
    direction = np.degrees(np.arctan2(lon_km, lat_km)) % 360
    
    # Convert speed to knots (1 knot = 1.852 km/h)
    speed_knots = speed_kmh / 1.852
    
    return speed_knots, direction

# Fill missing velocity for storms that need it
for sid in df['sid'].unique():
    storm = df[df['sid'] == sid]
    missing_mask = storm['storm_speed'].isna() | storm['storm_dir'].isna()
    
    if missing_mask.any():
        speed, direction = compute_velocity_from_positions(storm)
        df.loc[storm.index[missing_mask], 'storm_speed'] = speed[missing_mask].values
        df.loc[storm.index[missing_mask], 'storm_dir'] = direction[missing_mask].values


In [5]:
# Convert positions and velocities to metric coordinates (km) for Kalman filter
# This ensures unit consistency: state vector in km, velocities in km/6h

# Earth radius in km (approximate)
R_EARTH = 111.0  # km per degree latitude

def convert_to_metric_coordinates(df_storm):
    """Convert lat/lon to metric (x, y) coordinates in km relative to storm start"""
    df_storm = df_storm.sort_values('iso_time').copy()
    
    # Use first position as reference point
    lat_ref = df_storm['lat'].iloc[0]
    lon_ref = df_storm['lon'].iloc[0]
    
    # Convert to metric coordinates
    # x: east-west (longitude direction) in km
    # y: north-south (latitude direction) in km
    lat_rad = np.radians(df_storm['lat'])
    lon_rad = np.radians(df_storm['lon'])
    lat_ref_rad = np.radians(lat_ref)
    lon_ref_rad = np.radians(lon_ref)
    
    # Haversine-based conversion
    dlat = lat_rad - lat_ref_rad
    dlon = lon_rad - lon_ref_rad
    
    # Approximate metric coordinates (valid for small distances)
    # More accurate than simple scaling, accounts for latitude
    a = np.sin(dlat/2)**2 + np.cos(lat_ref_rad) * np.cos(lat_rad) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R_EARTH * np.degrees(c)
    
    # Direction from reference
    y_km = R_EARTH * np.degrees(dlat)
    x_km = R_EARTH * np.cos(lat_ref_rad) * np.degrees(dlon)
    
    return x_km.values, y_km.values, lat_ref, lon_ref

# Convert positions to metric coordinates for each storm
df['x_km'] = np.nan
df['y_km'] = np.nan
df['lat_ref'] = np.nan
df['lon_ref'] = np.nan

for sid in df['sid'].unique():
    storm_idx = df[df['sid'] == sid].index
    x_km, y_km, lat_ref, lon_ref = convert_to_metric_coordinates(df.loc[storm_idx])
    df.loc[storm_idx, 'x_km'] = x_km
    df.loc[storm_idx, 'y_km'] = y_km
    df.loc[storm_idx, 'lat_ref'] = lat_ref
    df.loc[storm_idx, 'lon_ref'] = lon_ref

# Compute velocities in km per 6 hours
# Calculate velocity as change in position per 6-hour period
def compute_metric_velocity(storm_data):
    """Compute velocity in km/6h from position changes"""
    storm_data = storm_data.sort_values('iso_time').copy()
    
    dt_hours = storm_data['iso_time'].diff().dt.total_seconds() / 3600
    dx_km = storm_data['x_km'].diff()
    dy_km = storm_data['y_km'].diff()
    
    # Convert to velocity in km per 6 hours
    # Normalize by actual time difference, then scale to 6-hour units
    vx_km_6h = (dx_km / dt_hours.replace(0, np.nan)) * 6
    vy_km_6h = (dy_km / dt_hours.replace(0, np.nan)) * 6
    
    # Fill first NaN with storm_speed if available
    if pd.isna(vx_km_6h.iloc[0]) and not pd.isna(storm_data['storm_speed'].iloc[0]):
        speed_kmh = storm_data['storm_speed'].iloc[0] * 1.852  # knots to km/h
        speed_km_6h = speed_kmh * 6
        direction = np.radians(storm_data['storm_dir'].iloc[0])
        vx_km_6h.iloc[0] = speed_km_6h * np.sin(direction)
        vy_km_6h.iloc[0] = speed_km_6h * np.cos(direction)
    
    return vx_km_6h.values, vy_km_6h.values

for sid in df['sid'].unique():
    storm_idx = df[df['sid'] == sid].index
    vx_km, vy_km = compute_metric_velocity(df.loc[storm_idx])
    df.loc[storm_idx, 'vx_km'] = vx_km
    df.loc[storm_idx, 'vy_km'] = vy_km

# Also keep degree-based velocities for backward compatibility (will be deprecated)
df['v_lat'] = df['storm_speed'] * 1.852 * 6 / 111.0 * np.cos(np.radians(df['storm_dir']))
df['v_lon'] = df['storm_speed'] * 1.852 * 6 / (111.0 * np.cos(np.radians(df['lat']))) * np.sin(np.radians(df['storm_dir']))


In [6]:
# Temporal features
df['storm_age_hours'] = df.groupby('sid')['iso_time'].transform(lambda x: (x - x.min()).dt.total_seconds() / 3600)
df['day_of_year'] = df['iso_time'].dt.dayofyear
df['month'] = df['iso_time'].dt.month


In [7]:
# Compute acceleration (change in velocity) for enhanced state representation
def compute_acceleration(storm_data):
    storm_data = storm_data.sort_values('iso_time').copy()
    dt = storm_data['iso_time'].diff().dt.total_seconds() / 3600
    
    dv_lat = storm_data['v_lat'].diff() / dt.replace(0, np.nan)
    dv_lon = storm_data['v_lon'].diff() / dt.replace(0, np.nan)
    
    return dv_lat, dv_lon

for sid in df['sid'].unique():
    storm_idx = df[df['sid'] == sid].index
    dv_lat, dv_lon = compute_acceleration(df.loc[storm_idx])
    df.loc[storm_idx, 'a_lat'] = dv_lat.values
    df.loc[storm_idx, 'a_lon'] = dv_lon.values


In [8]:
# Track curvature: measures how sharply storm is turning (indicates when linear assumption breaks)
def compute_track_curvature(storm_data):
    storm_data = storm_data.sort_values('iso_time').copy()
    
    ddir = storm_data['storm_dir'].diff()
    ddir = ddir.copy()
    ddir[ddir > 180] -= 360
    ddir[ddir < -180] += 360
    ddir = ddir.abs()
    
    speed = storm_data['storm_speed']
    curvature = ddir / (speed.replace(0, np.nan) + 1e-6)
    
    return curvature

for sid in df['sid'].unique():
    storm_idx = df[df['sid'] == sid].index
    df.loc[storm_idx, 'track_curvature'] = compute_track_curvature(df.loc[storm_idx]).values

# Latitude regime: affects storm motion characteristics
df['latitude_regime'] = pd.cut(df['lat'].abs(), bins=[0, 18, 28, 90], labels=[0, 1, 2], right=False).astype(float)

# Hemisphere indicator
df['hemisphere'] = (df['lat'] > 0).astype(int)

# Motion regime classification based on direction and speed
def classify_motion_regime(row):
    if pd.isna(row['storm_dir']) or pd.isna(row['storm_speed']):
        return np.nan
    
    speed = row['storm_speed']
    direction = row['storm_dir']
    
    if speed < 5:
        return 2
    elif 225 <= direction <= 315 or (direction <= 45) or (direction >= 315):
        return 0
    else:
        return 1

df['motion_regime'] = df.apply(classify_motion_regime, axis=1)

# Storm stage encoding from nature column
def encode_storm_stage(nature):
    if pd.isna(nature):
        return np.nan
    mapping = {'DS': 0, 'NR': 1, 'SS': 1, 'TS': 2, 'MX': 3, 'ET': 4}
    return mapping.get(nature, np.nan)

if 'nature' in df.columns:
    df['storm_stage'] = df['nature'].apply(encode_storm_stage)

# Landfall proximity features
if 'dist2land' in df.columns:
    df['is_approaching_land'] = (df['dist2land'] < 200).astype(int)
    df['land_gradient'] = df.groupby('sid')['dist2land'].diff() * -1  # Negative means approaching

# Beta-drift proxy: Coriolis-related drift (approximation)
df['beta_drift_lat'] = df['hemisphere'] * np.sin(np.radians(df['lat'].abs())) * 0.1
df['beta_drift_lon'] = df['hemisphere'] * np.sign(df['lat']) * np.cos(np.radians(df['lat'].abs())) * 0.05

# Smoothed velocity using 3-point moving average (reduces noise)
df['v_lat_smooth'] = df.groupby('sid')['v_lat'].transform(lambda x: x.rolling(window=3, center=True, min_periods=1).mean())
df['v_lon_smooth'] = df.groupby('sid')['v_lon'].transform(lambda x: x.rolling(window=3, center=True, min_periods=1).mean())

# Autoregressive motion features: recent motion averages
df['v_lat_avg_6h'] = df.groupby('sid')['v_lat'].transform(lambda x: x.rolling(window=2, min_periods=1).mean())
df['v_lon_avg_6h'] = df.groupby('sid')['v_lon'].transform(lambda x: x.rolling(window=2, min_periods=1).mean())
df['v_lat_avg_12h'] = df.groupby('sid')['v_lat'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df['v_lon_avg_12h'] = df.groupby('sid')['v_lon'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# Ensure agency columns exist (merge raw if missing)
needed_agency_cols = ["usa_agency", "wmo_agency", "usa_lat", "usa_lon", "basin"]
missing_agency_info = [c for c in needed_agency_cols if c not in df.columns]

if missing_agency_info:
    print(f"Missing agency cols: {missing_agency_info}")
    print("Loading raw IBTrACS to merge agency info...")
    
    def load_ibtracs_raw(path="ibtracs.ALL.list.v04r01.csv"):
        raw = pd.read_csv(path, skiprows=[1], low_memory=False)
        raw.columns = [c.lower() for c in raw.columns]
        raw = raw.replace(" ", np.nan)
        raw["iso_time"] = pd.to_datetime(raw["iso_time"])
        return raw
    
    df_raw = load_ibtracs_raw("ibtracs.ALL.list.v04r01.csv")
    pull_cols = ["sid", "iso_time", "basin", "subbasin", "usa_agency", "wmo_agency", "usa_lat", "usa_lon"]
    pull_cols = [c for c in pull_cols if c in df_raw.columns]
    df_agency = df_raw[pull_cols].copy()
    
    df = df.merge(df_agency, on=["sid", "iso_time"], how="left", suffixes=("", "_raw"))
    if "basin_raw" in df.columns:
        df["basin"] = df["basin"].fillna(df["basin_raw"])
        df.drop(columns=["basin_raw"], inplace=True)
    if "subbasin_raw" in df.columns and "subbasin" not in df.columns:
        df.rename(columns={"subbasin_raw": "subbasin"}, inplace=True)

# Fill missing basin using subbasin
if "subbasin" in df.columns:
    sub_to_basin = {"BB": "NI", "AS": "NI", "CP": "WP", "MM": "WP", "EA": "SP", "WA": "SP"}
    df["basin"] = df["basin"].fillna(df["subbasin"].map(sub_to_basin))

# Select authoritative agency tracks
def select_agency_tracks_fast(df):
    df = df.copy()
    preferred_by_basin = {
        "EP": ["hurdat_epa"],
        "NA": ["hurdat_atl"],
        "WP": ["jtwc_wp", "jtwc_cp", "jtwc_ep"],
        "NI": ["jtwc_io"],
        "SI": ["jtwc_sh", "jtwc_io"],
        "SP": ["jtwc_sh"],
        "SA": ["jtwc_sh"],
    }
    
    df["use_for_modeling"] = False
    df["modeling_agency"] = pd.NA
    
    if "usa_agency" not in df.columns:
        print("Warning: usa_agency missing. Using all rows.")
        df["use_for_modeling"] = True
        df["modeling_agency"] = "all"
        return df
    
    for basin, agencies in preferred_by_basin.items():
        mask = (df["basin"] == basin) & (df["usa_agency"].isin(agencies))
        df.loc[mask, "use_for_modeling"] = True
        df.loc[mask, "modeling_agency"] = df.loc[mask, "usa_agency"]
    
    # Overwrite lat/lon with authoritative agency positions where available
    if {"usa_lat", "usa_lon"}.issubset(df.columns):
        pos_mask = df["use_for_modeling"] & df["usa_lat"].notna() & df["usa_lon"].notna()
        df.loc[pos_mask, "lat"] = df.loc[pos_mask, "usa_lat"].astype(float)
        df.loc[pos_mask, "lon"] = df.loc[pos_mask, "usa_lon"].astype(float)
    
    return df

df = select_agency_tracks_fast(df)

# Build state columns dynamically (include metric columns if present)
metric_cols = ["x_km", "y_km", "vx_km", "vy_km", "lat_ref", "lon_ref"]
metric_cols_present = [c for c in metric_cols if c in df.columns]

base_state_cols = ["sid", "iso_time", "lat", "lon", "v_lat", "v_lon", "storm_speed", "storm_dir"]
state_cols = base_state_cols + metric_cols_present

if "usa_wind" in df.columns:
    state_cols.append("usa_wind")

# Build feature columns (only keep those that exist)
feature_cols = [
    "track_curvature", "latitude_regime", "hemisphere", "motion_regime",
    "storm_age_hours", "day_of_year", "month",
    "beta_drift_lat", "beta_drift_lon",
    "v_lat_smooth", "v_lon_smooth",
    "v_lat_avg_6h", "v_lon_avg_6h",
    "v_lat_avg_12h", "v_lon_avg_12h"
]

optional_features = ["storm_stage", "dist2land", "is_approaching_land", "land_gradient"]
feature_cols.extend([c for c in optional_features if c in df.columns])

# Extra columns for bookkeeping
extra_cols = ["basin", "nature", "use_for_modeling", "modeling_agency", "usa_agency"]
extra_cols = [c for c in extra_cols if c in df.columns]

# Build clean dataset
df_clean = df[state_cols + feature_cols + extra_cols].copy()
df_clean = df_clean.dropna(subset=["lat", "lon"]).reset_index(drop=True)

# Save full dataset with all engineered columns
df.to_pickle("hurricane_paths_processed_FULL.pkl")
print("Saved full feature df -> hurricane_paths_processed_FULL.pkl")

# Save clean dataset
df_clean.to_pickle("hurricane_paths_processed_CLEAN.pkl")
print("Saved clean modeling df -> hurricane_paths_processed_CLEAN.pkl")

# Save modeling-only dataset (authoritative-agency subset)
df_model_clean = df_clean[df_clean["use_for_modeling"]].copy()
df_model_clean = df_model_clean.sort_values(["sid", "iso_time"]).reset_index(drop=True)
df_model_clean.to_pickle("hurricane_paths_processed_MODEL.pkl")
print("Saved modeling-only df -> hurricane_paths_processed_MODEL.pkl")

print(f"\nState cols: {state_cols}")
print(f"Metric cols present: {metric_cols_present}")
print(f"df_clean rows: {len(df_clean):,}")
print(f"df_model_clean rows: {len(df_model_clean):,}")
print(f"storms total: {df_clean['sid'].nunique():,}")
print(f"storms modeling: {df_model_clean['sid'].nunique():,}")


Saved full feature df -> hurricane_paths_processed_FULL.pkl
Saved clean modeling df -> hurricane_paths_processed_CLEAN.pkl
Saved modeling-only df -> hurricane_paths_processed_MODEL.pkl

State cols: ['sid', 'iso_time', 'lat', 'lon', 'v_lat', 'v_lon', 'storm_speed', 'storm_dir', 'x_km', 'y_km', 'vx_km', 'vy_km', 'lat_ref', 'lon_ref', 'usa_wind']
Metric cols present: ['x_km', 'y_km', 'vx_km', 'vy_km', 'lat_ref', 'lon_ref']
df_clean rows: 721,960
df_model_clean rows: 170,106
storms total: 13,450
storms modeling: 6,360


In [9]:
# Validate data quality
print(f"Processed dataset: {len(df_clean):,} observations")
print(f"Unique storms: {df_clean['sid'].nunique():,}")
# Check for metric coordinates
metric_cols = ['x_km', 'y_km', 'vx_km', 'vy_km']
has_metric = all(col in df_clean.columns for col in metric_cols)

if has_metric:
    print(f"Missing values in metric state variables: {df_clean[metric_cols].isnull().sum().sum()}")
    print(f"Missing values in degree state variables: {df_clean[['lat', 'lon', 'v_lat', 'v_lon']].isnull().sum().sum()}")
else:
    print(f"Missing values in state variables: {df_clean[['lat', 'lon', 'v_lat', 'v_lon']].isnull().sum().sum()}")
print(f"Date range: {df_clean['iso_time'].min()} to {df_clean['iso_time'].max()}")


Processed dataset: 721,960 observations
Unique storms: 13,450
Missing values in metric state variables: 0
Missing values in degree state variables: 0
Date range: 1842-10-25 03:00:00 to 2025-11-23 00:00:00


In [10]:
# Save processed dataset
df_clean.to_pickle("hurricane_paths_processed.pkl")
df_clean.to_csv("hurricane_paths_processed.csv", index=False)
